<br><br><br><br>
<h1>01. Exploring Marginal Distributions of stock data returns</h1>

<p>Applying heteroskedastic time series models to asset data and retrieving the marginal distribution.</p>

<br><br><br><br>
<h2>0. Document setup</h2>

In [6]:
import sys
from pathlib import Path

# Add project root so we can import from src/
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

<br><br><br><br>
<h2>1. Data preparation</h2>

In [10]:
from src.data_handlers.yfinance_data_loader import load_prices

prices_appl = load_prices("AAPL", "2015-01-01", "2024-01-01")

prices_ge = load_prices("GE", "2015-01-01", "2024-01-01")


/Users/isakdrougge/GitHub-Repos/applications-copula-methods-in-finance/src/data_handlers/yfinance_data_loader.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(file_path, index_col=0, parse_dates=True)
/Users/isakdrougge/GitHub-Repos/applications-copula-methods-in-finance/src/data_handlers/yfinance_data_loader.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(file_path, index_col=0, parse_dates=True)


In [11]:
prices_ge.head()

,Close,High,Low,Open,Volume
Price,,,,,
Ticker,GE,GE,GE,GE,GE
Date,NaN,NaN,NaN,NaN,NaN
2015-01-02,102.69725036621094,104.33647393694966,101.91862210589264,103.8447042561435,8538620
2015-01-05,100.81212615966797,102.49232641387374,100.48428411707955,102.49232641387374,8770735
2015-01-06,98.64015197753906,100.68917770148151,98.06642842825268,100.23839165077659,13304295


In [12]:

prices_appl.head()

,Close,High,Low,Open,Volume
Price,,,,,
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,NaN,NaN,NaN,NaN,NaN
2015-01-02,24.237546920776367,24.705316407099893,23.798596738180272,24.694231148773138,212818400
2015-01-05,23.554738998413086,24.0867993234754,23.368518814896508,24.006990190147395,257142000
2015-01-06,23.556962966918945,23.816341876980253,23.195604375837014,23.619036378758544,263188400


<br><br><br><br>
<h2>text</h2>